## The following code uses the Pythonspaudiopy package
- Docs: https://spaudiopy.readthedocs.io/en/latest/index.html
- GitHub: https://github.com/chris-hld/spaudiopy

## Tools
We use three parameters to locate an HRTF:
1. *Azimuth*: angle between position and sound location on the $xy$-plane
2. *Elevation*: angle between position and sound location on the $xz$-plane
3. *Time* or *Frequency*: time period or frequency of emitted sound w.r.t. actual position

Math tools:
- *Haversine distance*: Consider two points $x_1$ and $x_2$ on a sphere with respective latitudes and longitudes $(\varphi_1,\varphi_2)$ and $(\theta_1,\theta_2)$. The Haversine distance $D(x_1,x_2)$ is the angular distance between them on the surface of the sphere given by $$D(x_1,x_2)=2\arcsin\sqrt{\sin^2\left(\frac{\varphi_2-\varphi_1}{2}\right)+\cos x_1\cos y_1\sin^2\left(\frac{\theta_2-\theta_1}{2}\right)}.$$ We use this distance when wanting to find the closest HRIR point from a grid.

## Imports

In [ ]:
# Check spaudiopy.sig functions to open MONO signal and play with HRIRs
import spaudiopy as spa
from spaudiopy.sig import MonoSignal as ms
from spaudiopy.sig import MultiSignal as stereo 
import spaudiopy.process as sproc

import scipy.signal
from scipy.io import wavfile
from IPython.display import Audio

fs=44100 # sampling rate
fs2=96000
import numpy as np

## Simple spatialisation with delay

In [ ]:
def get_stereo_tracks(music_file):
    return stereo.from_file(music_file).get_signals()

def delay_signal(s, delay):
    _len_sample = len(s)
    d_arr = np.zeros(_len_sample, dtype=int)
    d_arr[int(fs*delay)] = 1
    return scipy.signal.convolve(s, d_arr)[:_len_sample]

def spatialize_signal(s, delay):
    _len_sample = len(s)
    d_arr = np.zeros(_len_sample, dtype=int)
    d_arr[0] = 1
    d_arr[int(fs*delay)] = 1
    return scipy.signal.convolve(s, d_arr)[:_len_sample]


In [ ]:
MS_DELAY = 25 # delay in [ms] to create surrounding effect

def spatialise_sound(music_file, left=False):
    """Create a spatialised version of the given input file
        @param music_file: stereo music file
        @param left: creates the delay to the left or the right given this value
    """
    s1,s2 = get_stereo_tracks(music_file)
    if left:
        delayed_s1 = delay_signal(s1, MS_DELAY / 1000)
        return stereo([delayed_s1, s2], fs=fs)
    delayed_s2 = delay_signal(s2, MS_DELAY / 1000)
    return stereo([s1, delayed_s2], fs=fs)

In [ ]:
def spatialise_sound_three(music_file, left=False):
    """Create a spatialised version of the given input file
        @param music_file: stereo music file
        @param left: creates the delay to the left or the right given this value
    """
    s1,s2 = get_stereo_tracks(music_file)
    if left:
        delayed_s1 = delay_signal(s1, MS_DELAY / 1000)
        return stereo([delayed_s1, s1, s2], fs=fs)
    delayed_s2 = delay_signal(s2, MS_DELAY / 1000)
    return stereo([s1, s2, delayed_s2], fs=fs)

In [ ]:
s = spatialise_sound("ocean_eyes.wav")
s.save("ocean_eyes_spatial_2tracks.wav")
display(Audio(s, rate=fs))

In [ ]:
s2 = spatialise_sound_three("ocean_eyes.wav")
s2.save("ocean_eyes_spatial_3tracks.wav")
display(Audio(s2, rate=fs))

## Reverb for increased spatial effect
Create simultaneous small delay effects and combine them together in a new signal.

In [ ]:
# TODO: WORK IN PROGRESS
def reverb(music_file):
    s1,s2 = get_stereo_tracks(music_file)
    tracks = [s1]
    for _ in range(5):
        delayed_s2 = delay_signal(s2, 5 / 1000)
        tracks.append(delayed_s2)
        s2 = delayed_s2
    return stereo(tracks, fs=fs)

s_rev = reverb("ocean_eyes.wav")
display(Audio(s_rev, rate=fs))

In [ ]:
# TODO: WORK IN PROGRESS
def echo(s):
    _filter = [1]
    for i in range(1, len(s)):
        _filter.append(_filter[-1]*0.5)
    return _filter

def successive_echo(s):
    _filter = [1]
    for i in range(1, len(s)):
        if i % fs/2 == 0:
            _filter.append(1)
        else:
            _filter.append(_filter[-1]*0.99)
    return _filter

def successive_delays(s, delay):
    return scipy.signal.convolve(s, echo(s))[:len(s)]

def reverb_v2(music_file):
    s1,s2 = get_stereo_tracks(music_file)
    delayed_s1 = delay_signal(s1, MS_DELAY / 1000)
    delayed_s2 = successive_delays(s2, MS_DELAY / 1000)
    return stereo([s1, s2, delayed_s2], fs=fs)

s_rev2 = reverb_v2("ocean_eyes.wav")
display(Audio(s_rev2, rate=fs))

## Simple sound localisation

In [ ]:
def attenuate(sig, direction):
    return [e*np.cos(direction/4) for e in sig]

def localize_sound(stereo_music_file, direction, left=True):
    sample = stereo.from_file(stereo_music_file)
    s1, s2 = sample.get_signals()
    sample_song_mono = fusion([s1,s2])
    
    maxmono = max(sample_song_mono)
    mono_norm = []
    for i in range(len(sample_song_mono)):
        mono_norm.append(sample_song_mono[i]/(maxmono+1))
        
    delay = np.zeros(len(mono_norm), dtype=int)
    delay[int(fs*compute_delay_from_angle(direction))] = 1
    delayed = scipy.signal.convolve(mono_norm, delay)[:len(mono_norm)]
    
    if left:
        return stereo([mono_norm, attenuate(delayed, direction)], fs=44100) # sound to the left
    return stereo([attenuate(delayed, direction), mono_norm], fs=44100) # sound to the right

display(Audio(localize_sound("ocean_eyes.wav", np.pi/4), rate=44100))

## Pyroomacoustics tests for localisation

In [ ]:
import pyroomacoustics as pra
import matplotlib.pyplot as plt

In [ ]:
#signal = ms.from_file("groove.wav").signal
fs, signal = wavfile.read("groove.wav")

# Sound source part
room = pra.ShoeBox([20,20])
room.add_source([10, 10], signal=signal)

# Microphones part
#R = pra.circular_2D_array(center=[2.,2.], M=6, phi0=0, radius=0.1)
#room.add_microphone_array(pra.MicrophoneArray(R, room.fs))
R = np.array([[9, 11], [8, 8]])  # [[x], [y]]
room.add_microphone(R)

room.simulate()
fig, ax = room.plot()

In [ ]:
def spatialise_signals(s1, s2):
    delayed_s2 = delay_signal(s2, MS_DELAY / 1000)
    return stereo([s1, delayed_s2], fs=fs)

#snd = spatialise_signals(room.mic_array.signals[0,:], room.mic_array.signals[1,:])
snd = stereo([room.mic_array.signals[0,:], room.mic_array.signals[1,:]], fs=fs)
display(Audio(snd, rate=fs))

In [ ]:
song = stereo.from_file("Police_Roxanne/The-Police-Roxanne.wav")
bass = stereo.from_file("Police_Roxanne/bass.wav")
drums = stereo.from_file("Police_Roxanne/drums.wav")
other = stereo.from_file("Police_Roxanne/other.wav")
vocals = stereo.from_file("Police_Roxanne/vocals.wav")

song1, song2 = song.get_signals()
b1, b2 = bass.get_signals()
d1, d2 = drums.get_signals()
o1, o2 = other.get_signals()
v1, v2 = vocals.get_signals()

mono_song = fusion([trim(song1), trim(song2)])
mono_bass = fusion([trim(b1), trim(b2)])
mono_drums = fusion([trim(d1), trim(d2)])
mono_other = fusion([trim(o1), trim(o2)])
mono_vocals = fusion([trim(v1), trim(v2)])

In [ ]:
# Sound source part
room = pra.ShoeBox([20,20])
#room.add_source([10, 10], signal=mono_song)
room.add_source([17, 8], signal=mono_bass)
room.add_source([10, 17], signal=mono_drums)
room.add_source([3, 8], signal=mono_other)
#room.add_source([10, 12], signal=mono_vocals)

# Microphones part
#R = pra.circular_2D_array(center=[2.,2.], M=6, phi0=0, radius=0.1)
#room.add_microphone_array(pra.MicrophoneArray(R, room.fs))
R = np.array([[9, 11], [8, 8]])  # [[x], [y]]
room.add_microphone(R)

room.simulate()
fig, ax = room.plot()

In [ ]:
def shut_the_fuck_up(sound):
    return [e*0.2 for e in sound]

left = fusion([trim(room.mic_array.signals[0,:]), shut_the_fuck_up(spatialize(mono_vocals))])
right = fusion([trim(room.mic_array.signals[1,:]), shut_the_fuck_up(mono_vocals)])

#final = stereo([room.mic_array.signals[0,:], room.mic_array.signals[1,:]], fs=fs)
final = stereo([left, right], fs=fs)
display(Audio(final, rate=fs))

## Localise instruments

In [ ]:
avg_ear_distance = 0.21 # 21cm
speed_of_sound = 343 # in m/s

def trim(sig):
    return sig[:44100*30]

def compute_delay_from_angle(rad):
    return avg_ear_distance * np.sin(rad) / speed_of_sound

def fusion(signals):
    s = []
    mean = np.mean(signals, axis=0)
    for i in range(len(signals[0])):
        s.append(np.mean(mean[i]))
    return s

def delay(sig, direction):
    delay = np.zeros(len(sig), dtype=int)
    delay[int(fs*compute_delay_from_angle(direction))] = 1
    
    return scipy.signal.convolve(sig, delay)[:len(sig)]

def spatialize(sig):
    delay = np.zeros(len(sig), dtype=int)
    delay[int(fs*0.025)] = 1
    return scipy.signal.convolve(sig, delay)[:len(sig)]


song = stereo.from_file("Police_Roxanne/The-Police-Roxanne.wav")
bass = stereo.from_file("Police_Roxanne/bass.wav")
drums = stereo.from_file("Police_Roxanne/drums.wav")
other = stereo.from_file("Police_Roxanne/other.wav")
vocals = stereo.from_file("Police_Roxanne/vocals.wav")

song1, song2 = song.get_signals()
b1, b2 = bass.get_signals()
d1, d2 = drums.get_signals()
o1, o2 = other.get_signals()
v1, v2 = vocals.get_signals()

mono_song = fusion([trim(song1), trim(song2)])
mono_bass = fusion([trim(b1), trim(b2)])
mono_drums = fusion([trim(d1), trim(d2)])
mono_other = fusion([trim(o1), trim(o2)])
mono_vocals = fusion([trim(v1), trim(v2)])

left = trim(fusion([mono_song, mono_bass, mono_drums, delay(mono_other, np.pi/2), spatialize(mono_vocals)]))
right = trim(fusion([mono_song, mono_bass, delay(mono_drums, np.pi/2), mono_other, mono_vocals]))

display(Audio(stereo([left, right], fs=fs), rate=fs))

## Frequency plots
We wish to visualise the main differences between the two frequency plots of both the original stereo file and the generated spatialised one.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def plot_freqs(wav_file, start, end):
    fs, data = wavfile.read(wav_file)
    data = data[:,0]
    plt.figure()
    data_to_plot = data[fs*start:fs*end]
    plt.plot(data_to_plot)
    plt.xlabel('Sample Index')
    plt.ylabel('Amplitude')
    plt.title('Waveform of ' + wav_file)
    plt.show()
    return data_to_plot

In [ ]:
f_ster = plot_freqs('ocean_eyes.wav', 30, 31)
f_spat = plot_freqs('ocean_eyes_spatialised.wav', 30, 31)